In [1]:
import pandas as pd
import numpy as np
import pickle
from collections import Counter, defaultdict

In [2]:
import sys
import os
def add_sys_path(p):
    p = os.path.abspath(p)
    if p not in sys.path:
        sys.path.append(p)

add_sys_path('..')

In [3]:
import evaluate

In [4]:
import data_split

In [6]:
from evaluate import read_dataset

In [11]:
import json

In [69]:
from tqdm.auto import tqdm, trange

# Чтение и сплит данных

In [13]:
v_ds = read_dataset('../data/training_data/training_verbs.tsv',  lambda x: json.loads(x))

In [16]:
v_ds['АМНИСТИРОВАТЬ']

[['115444-V', '120034-V'], ['117915-V', '135001-V']]

In [20]:
train_share = 0.8
dev_share = 0.02
test1_share = 0.02
test2_share = 0.02
hid_share = 0.14

assert train_share + dev_share + test1_share + test2_share + hid_share == 1

In [21]:
from data_split import hash_float

In [31]:
train, dev, test1, test2, hid = {}, {}, {} , {}, {}

for k, v in v_ds.items():
    h = hash_float(k)
    if h <= train_share:
        train[k] = v
    elif h <= train_share + dev_share:
        dev[k] = v
    elif h <= train_share + dev_share + test1_share:
        test1[k] = v
    elif h <= train_share + dev_share + test1_share + test2_share:
        test2[k] = v
    else:
        hid[k] = v

In [146]:
forbidden_words = {w for s in [dev, test1, test2, hid] for w in s.keys()}

# Применение бейзлайна

In [192]:
import my_knn
from importlib import reload
reload(my_knn)

<module 'my_knn' from 'C:\\Users\\ddale\\YandexDisk\\code\\NLP\\taxonomy-enrichment\\experiments\\my_knn.py'>

In [33]:
import gensim
# this is araneum 2018 ft model from rusvectores
ft = gensim.models.fasttext.FastTextKeyedVectors.load(
    'C:/Users/ddale/Downloads/NLP/rusvectores/model.model'
)

In [201]:
embedder = my_knn.SentenceEmbedder(ft=ft, n=300, normalize_word=True)
print(embedder('привет как дела').shape)

(300,)


In [42]:
import xmltodict
with open('../data/ruwordnet/synsets.V.xml', 'r', encoding='utf-8') as f:
    synsets_v_raw = xmltodict.parse(f.read(), process_namespaces=True)

In [50]:
with open('../data/ruwordnet/synset_relations.V.xml', 'r', encoding='utf-8') as f:
    rel_v_raw = xmltodict.parse(f.read(), process_namespaces=True)

Without fobidding, list is 49k, with forbidding it is 28k only

In [153]:
verbs_storage = my_knn.SynsetStorage.construct(synsets_v_raw, forbidden_words=forbidden_words)

number of texts: 32822
forbidden senses are 1390
numer of ids 7521 long list is 28657


In [154]:
rel_df = my_knn.make_rel_df(rel_v_raw, verbs_storage.id2synset)

In [155]:
rel_df.head()

,@parent_id,@child_id,@name,parent,child
0,4223-V,4223-N,POS-synonymy,ЛИВЕНЬ,None
1,4223-V,4223-A,POS-synonymy,ЛИВЕНЬ,None
2,4223-V,5068-N,domain,ЛИВЕНЬ,None
3,129120-V,129121-V,hyponym,"БЫТЬ ИЗВЕСТНЫМ, СЛЫТЬ",СЛАВИТЬСЯ (ПОЛЬЗОВАТЬСЯ ИЗВЕСТНОСТЬЮ)
4,129120-V,148420-V,hypernym,"БЫТЬ ИЗВЕСТНЫМ, СЛЫТЬ",ОТНОШЕНИЯ МЕЖДУ ЛЮДЬМИ


In [165]:
verb_rel_storage = my_knn.RelationStorage(forbidden_id=verbs_storage.forbidden_id)
verb_rel_storage.construct_relations(rel_df)


5752
186
5
7480


In [166]:
len(verbs_storage.texts_long)

28657

In [209]:
import random

In [252]:
from pymorphy2 import MorphAnalyzer

morphAnalyzer = MorphAnalyzer()

def word2pos(w):
    parses = morphAnalyzer.parse(w)
    if not parses:
        return None
    parse = parses[0]
    if not parse.tag:
        return None
    if not parse.tag.POS:
        return None
    return parse.tag.POS

In [365]:
s = random.sample(verbs_storage.texts_long, 5)
for t in s:
    print(t)
    print([word2pos(w) for w in my_knn.tokenize(t)])

ШАТНУТЬСЯ
['INFN']
ВЫЗВАТЬСЯ ; ВЫЗВАТЬСЯ ДОБРОВОЛЬНО ; ВЫЗЫВАТЬСЯ ; ДОБРОВОЛЬНО ВЗЯТЬСЯ ; ДОБРОВОЛЬНО ВЫЗВАТЬСЯ ; СВЯЗАТЬСЯ
['INFN', 'INFN', 'ADVB', 'INFN', 'ADVB', 'INFN', 'ADVB', 'INFN', 'INFN']
НАРУШЕНИЕ ВОИНСКОЙ ДИСЦИПЛИНЫ
['NOUN', 'ADJF', 'NOUN']
АРЕНДОВАТЬ НЕДВИЖИМОСТЬ
['INFN', 'NOUN']
ПОСЛЫШАТЬСЯ
['INFN']


In [366]:
embedder = SentenceEmbedder(ft=ft, n=300, normalize_word=True, pos_weights={'INFN': 1.0, 'PREP': 0.1})

In [367]:
embedder.get_word_weights(my_knn.tokenize('играть в гольф'))

[1.0, 0.1, 1]

In [346]:
vecs = np.stack([embedder(t) for t in tqdm(verbs_storage.texts_long) ])

In [347]:
from sklearn.neighbors import KDTree
tree = KDTree(vecs)

In [348]:
hypos = my_knn.hypotheses_knn('ковырять', index=tree, text2vec=embedder, synset_storage=verbs_storage, rel_storage=verb_rel_storage)
hypos

['111845-V',
 '111846-V',
 '106701-V',
 '115889-V',
 '115208-V',
 '127537-V',
 '107094-V',
 '115188-V',
 '111888-V',
 '125754-V']

In [349]:
for id in hypos:
    print(verbs_storage.id2synset[id]['@ruthes_name'])

ПРОНИКНУТЬ ВНУТРЬ
ПОПАСТЬ (ОКАЗАТЬСЯ, ОЧУТИТЬСЯ)
ДОСТИЧЬ, ДОБИТЬСЯ
УЯЗВИТЬ
ОСКОРБИТЬ
ЕХИДНИЧАТЬ, ЯЗВИТЬ
ПРОИЗНЕСТИ, ВЫГОВОРИТЬ, ПРОГОВОРИТЬ
ОБЪЕСТЬ С КРАЕВ
КУСАТЬ ЗУБАМИ
ВОНЗИТЬ (ВОТКНУТЬ ОСТРИЕМ)


In [350]:
# проверка, что сравнение само с собой всегда получает скор 1
dev_pred = {k: [v0 for val in vals for v0 in val] for k, vals in dev.items()}
mean_ap, mean_rr = evaluate.get_score(dev, dev_pred, k=10)
print(mean_ap, mean_rr)

1.0 1.0


In [351]:
#texts = ttest_test1.one_text.drop_duplicates()
texts = sorted(dev.keys())
small_hypos = {
    txt: 
    my_knn.hypotheses_knn(
        txt, 
        index=tree, text2vec=embedder, synset_storage=verbs_storage, rel_storage=verb_rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.5,
    )  
    for txt in tqdm(texts)
}


Когда я выкинул из синсетов запрещённое, получаю 21% 27%, а если ещё из отношений, то 20% 26%. 

Хотя на паблик тесте вообще 24% 27%. 

Вывод: вообще-то перед сабмишнами надо пересчитываться на полной выборке. 

In [352]:
mean_ap, mean_rr = evaluate.get_score(dev, small_hypos, k=10)
print(mean_ap, mean_rr)

0.20307894783565925 0.2682939171194206


```
0.20069866126913108 0.26436294875892197 - baseline
0.19999533929903057 0.2675482049643124 - not normalize word vecs (inconclusive)

0.18938824970704168 0.24604239906253328 - baseline + weight{verb = 3}
0.19966265402507014 0.2668610844785341 - baseline + weight{verb=1.1}
0.20093569120414756 0.2669250026632577 - baseline + weight{prep=0.8} (definitely helps!)
0.20307894783565925 0.2682939171194206 - baseline + weight{prep=0.1} (we'll keep it this way)
```

### частные случаи

Проверяю, что может быть не так

In [486]:
word = random.choice(list(dev.keys()))
print(word)

print(dev[word])
print([
    [verbs_storage.id2synset[id]['@ruthes_name'] for id in ids]
    for ids in dev[word]
])

ЗАВОЛОЧЬ
[['107325-V', '108830-V']]
[['ПОКРЫТЬ ПОВЕРХНОСТЬ', 'НАКРЫТЬ, ПОКРЫТЬ СВЕРХУ']]


In [487]:
for h in small_hypos[word]:
    print(h, verbs_storage.id2synset[h]['@ruthes_name'])

107454-V ПОДНЯТЬСЯ (ПЕРЕМЕСТИТЬСЯ ВВЕРХ)
106587-V ДВИЖЕНИЕ, ПЕРЕМЕЩЕНИЕ
106629-V ИЗМЕНИТЬСЯ, ИЗМЕНЕНИЕ
113780-V ПОКРЫТЬСЯ НА ПОВЕРХНОСТИ
111001-V ВИДНЕТЬСЯ
115583-V НАПОЛНИТЬСЯ
111845-V ПРОНИКНУТЬ ВНУТРЬ
111085-V ОПУСТИТЬСЯ ВНИЗ
123600-V ВЗДЫМАТЬСЯ
110915-V ВОЛОЧИТЬ, ТАЩИТЬ ВОЛОКОМ


In [446]:
for h in my_knn.hypotheses_knn(
        'ОБМОЛОТИТЬ', 
        index=tree, text2vec=embedder, synset_storage=verbs_storage, rel_storage=verb_rel_storage,
        decay=3,  k=100,grand_mult=0.5,
    ) :
    print(h, verbs_storage.id2synset[h]['@ruthes_name'])

106501-V ЗАНЯТИЕ, ДЕЯТЕЛЬНОСТЬ
106631-V ИЗМЕНИТЬ, СДЕЛАТЬ ИНЫМ
110790-V ПРИКРЕПИТЬ, ПРИДЕЛАТЬ
106534-V ОБРАБАТЫВАТЬ, ПОДВЕРГАТЬ ОБРАБОТКЕ
107417-V ОТДЕЛИТЬ ЧАСТЬ ОТ ЦЕЛОГО
112813-V ПОСАДКА РАСТЕНИЙ
112737-V ЗАНЯТЬ ВСЕ ПРОСТРАНСТВО
151674-V ОБРАБОТКА ПОЧВЫ
115184-V ВСКОПАТЬ
106698-V ПОМЕСТИТЬ, НАЙТИ МЕСТО


Заметные проблемы:
* композиционность ("вывести из эксплуатации", "пробиться через препятствие", "давать на лапу") - ???
* не основной смысл ("блестеть") - нужна диверсификация ответа по смыслам?
* редкий вид слова ("обмолачивать - обмолотить") - добавлять в кандидатов разные формы

In [448]:
for sense in verbs_storage.word2sense[word]:
    print(sense, verbs_storage.id2synset[sense]['@ruthes_name'])

114124-V ОБМОЛОТ ЗЕРНА


# Оценка применения

In [452]:
full_syn_storage, full_rel_storage, full_rel_df = my_knn.prepare_storages(
    synsets_filename='../data/ruwordnet/synsets.V.xml',
    relations_filename='../data/ruwordnet/synset_relations.V.xml',
    forbidden_words=set()
)

number of texts: 32822
forbidden senses are 0
numer of ids 7521 long list is 49002



7408
218
5
10317


In [453]:
embedder = SentenceEmbedder(ft=ft, n=300, normalize_word=True, pos_weights={'INFN': 1.0, 'PREP': 0.1}, default_weight=0.5)

In [454]:
full_vecs = np.stack([embedder(t) for t in tqdm(full_syn_storage.texts_long) ])
full_tree = KDTree(full_vecs)

In [455]:
public_test_verbs = pd.read_csv('../data/public_test/verbs_public.tsv', header=None)
public_test_verbs.columns = ['text']

In [456]:
public_test_hypos = {
    txt: 
    my_knn.hypotheses_knn(
        txt, 
        index=full_tree, 
        text2vec=embedder, 
        synset_storage=full_syn_storage, rel_storage=full_rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.5,
    )  
    for txt in tqdm(public_test_verbs.text)
}

In [457]:
sub = my_knn.dict2submission(public_test_hypos, full_syn_storage.id2synset)
sub.head(15)

,noun,result,result_text
0,АБСОЛЮТИЗИРОВАТЬ,106501-V,"ЗАНЯТИЕ, ДЕЯТЕЛЬНОСТЬ"
1,АБСОЛЮТИЗИРОВАТЬ,7237-V,УПРАВЛЕНЧЕСКАЯ ДЕЯТЕЛЬНОСТЬ
2,АБСОЛЮТИЗИРОВАТЬ,141697-V,ПРЕДСТАВИТЬ В ВИДЕ
3,АБСОЛЮТИЗИРОВАТЬ,106629-V,"ИЗМЕНИТЬСЯ, ИЗМЕНЕНИЕ"
4,АБСОЛЮТИЗИРОВАТЬ,106607-V,"УПРАВЛЯТЬ, РУКОВОДИТЬ"
5,АБСОЛЮТИЗИРОВАТЬ,7936-V,РЕФОРМА
6,АБСОЛЮТИЗИРОВАТЬ,118698-V,"ДЕЙСТВИЕ, ЦЕЛЕНАПРАВЛЕННОЕ ДЕЙСТВИЕ"
7,АБСОЛЮТИЗИРОВАТЬ,119125-V,"РАСПРОСТРАНИТЬСЯ, ПОЛУЧИТЬ РАСПРОСТРАНЕНИЕ"
8,АБСОЛЮТИЗИРОВАТЬ,123550-V,"УТОЧНИТЬ, СДЕЛАТЬ ТОЧНЕЕ"
9,АБСОЛЮТИЗИРОВАТЬ,135851-V,ПРИДАТЬ СВОЙСТВО


In [458]:
sub.to_csv('results/verbs_v3_all_small_weight.tsv', sep='\t', encoding='utf-8', header=None, index=None)

Проверка на идентичность с предыдущей заливкой

In [183]:
prev = pd.read_csv('results/verbs_v0_strange.tsv', sep='\t', encoding='utf-8', header=None)
prev.columns = sub.columns

In [191]:
assert prev.shape == sub.shape
assert (prev.noun == sub.noun).all()
assert (prev.result == sub.result).all()

# Финальная заливка

In [138]:
private_test_verbs = pd.read_csv('../data/private_test/verbs_private.tsv', header=None)
private_test_verbs.columns = ['text']

In [140]:
private_test_hypos = {
    txt: 
    my_knn.hypotheses_knn(
        txt, 
        index=tree, text2vec=embedder, synset_storage=verbs_storage, rel_storage=verb_rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.5,
    )  
    for txt in tqdm(private_test_verbs.text)
}

In [141]:
sub = my_knn.dict2submission(private_test_hypos, verbs_storage.id2synset)

In [142]:
sub

,noun,result,result_text
0,АДСОРБИРОВАТЬ,111097-V,ИЗМЕНЕНИЕ АГРЕГАТНОГО СОСТОЯНИЯ
1,АДСОРБИРОВАТЬ,106629-V,"ИЗМЕНИТЬСЯ, ИЗМЕНЕНИЕ"
2,АДСОРБИРОВАТЬ,106631-V,"ИЗМЕНИТЬ, СДЕЛАТЬ ИНЫМ"
3,АДСОРБИРОВАТЬ,106950-V,"ПРЕВРАТИТЬ (ПРИДАТЬ ИНОЙ ВИД, КАЧЕСТВО)"
4,АДСОРБИРОВАТЬ,106501-V,"ЗАНЯТИЕ, ДЕЯТЕЛЬНОСТЬ"
...,...,...,...
3495,ЮМОРИТЬ,116284-V,"ВЫСМЕЯТЬ, ОСМЕЯТЬ"
3496,ЮМОРИТЬ,118698-V,"ДЕЙСТВИЕ, ЦЕЛЕНАПРАВЛЕННОЕ ДЕЙСТВИЕ"
3497,ЮМОРИТЬ,116285-V,НАСМЕХАТЬСЯ
3498,ЮМОРИТЬ,115102-V,МУЧИТЬ


In [143]:
sub.to_csv('results/private_verbs_v0_strange.tsv', sep='\t', encoding='utf-8', header=None, index=None)
sub.head(15)

,noun,result,result_text
0,АДСОРБИРОВАТЬ,111097-V,ИЗМЕНЕНИЕ АГРЕГАТНОГО СОСТОЯНИЯ
1,АДСОРБИРОВАТЬ,106629-V,"ИЗМЕНИТЬСЯ, ИЗМЕНЕНИЕ"
2,АДСОРБИРОВАТЬ,106631-V,"ИЗМЕНИТЬ, СДЕЛАТЬ ИНЫМ"
3,АДСОРБИРОВАТЬ,106950-V,"ПРЕВРАТИТЬ (ПРИДАТЬ ИНОЙ ВИД, КАЧЕСТВО)"
4,АДСОРБИРОВАТЬ,106501-V,"ЗАНЯТИЕ, ДЕЯТЕЛЬНОСТЬ"
5,АДСОРБИРОВАТЬ,112139-V,ОЧИСТКА ОТ ПРИМЕСЕЙ
6,АДСОРБИРОВАТЬ,106534-V,"ОБРАБАТЫВАТЬ, ПОДВЕРГАТЬ ОБРАБОТКЕ"
7,АДСОРБИРОВАТЬ,112136-V,ПРОПУСТИТЬ (ДАТЬ ПРОНИКНУТЬ)
8,АДСОРБИРОВАТЬ,107417-V,ОТДЕЛИТЬ ЧАСТЬ ОТ ЦЕЛОГО
9,АДСОРБИРОВАТЬ,145451-V,СЕПАРАЦИЯ СМЕСИ


# Гипотеза с n-граммами

http://www.ruscorpora.ru/new/corpora-freq.html

In [465]:
ngrams_path = 'C:/Users/ddale/Downloads/NLP/ruscorpora_ngrams'

In [470]:
import os
import pandas as pd

In [473]:
p = ngrams_path

grams_lines = {}
for fn in os.listdir(p):
    if not fn.endswith('txt'):
        continue
    print(fn)
    f0 = int(fn[0])
    with open(os.path.join(p, fn), encoding='utf-8') as f:
        grams_lines[f0] = [l.split('\t') for l in f.readlines()]

1grams-3.txt
2grams-3.txt
3grams-3.txt
4grams-2.txt
5grams-2.txt


In [474]:
def get_ngram(l):
    l = [w.strip() for w in l[1:]]
    return '_'.join([w for w in l if w])

In [475]:
grams_freq = {
    n: {
        get_ngram(l): int(l[0])
        for l in d
    }
    for n, d in grams_lines.items()
}

Из биграм мы узнаём, что "захламленной" может быть комната или квартира. БОльшие n не сообщают ничего

In [484]:
for k, v in grams_freq[2].items():
    if 'захламл' in k:
        print(k, v)

был_захламлен 5
захламленной_комнате 5
в_захламленной 4
в_захламленном 4
в_захламленный 4
захламленную_комнату 4
по_захламленному 4
была_захламлена 3
захламленной_квартире 3
захламленности_и 3


Фасттекст умнее - он знает, что это слово похоже на замусорять, загромождивать и загаживать, что бы это ни значило :)

In [485]:
ft.most_similar('захламлять')

[('захламлать', 0.9291685819625854),
 ('захламливать', 0.9029788970947266),
 ('захламленный', 0.8950067758560181),
 ('захламленность', 0.7632160186767578),
 ('замусорять', 0.6844406127929688),
 ('загромождивать', 0.6693085432052612),
 ('загаживать', 0.6531472206115723),
 ('загромождать', 0.6455291509628296),
 ('замусорить', 0.6451689600944519),
 ('задымлять', 0.642647385597229)]